In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import featurewiz as gwiz

In [6]:
data = pd.read_csv('dataset.csv')
data.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,...,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934


In [7]:
X = data.drop(['SMILES', 'Name'], axis=1)
y = data[['SMILES', 'Name']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape


((7985, 24), (1997, 24))

In [12]:
df_obj = data.select_dtypes(['string'])
print (df_obj)

data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
print (data)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[9982 rows x 0 columns]
        ID                                               Name   
0      A-3         N,N,N-trimethyloctadecan-1-aminium bromide  \
1      A-4                           Benzo[cd]indol-2(1H)-one   
2      A-5                               4-chlorobenzaldehyde   
3      A-8  zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...   
4      A-9  4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...   
...    ...                                                ...   
9977  I-84                                         tetracaine   
9978  I-85              

In [13]:
#fit a linear regression model on the original train set and evaluate its performance on the test set:

# fit a linear regression model on the original train set
model = LinearRegression()
model.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = model.predict(X_test)
r2_score_before = r2_score(y_test, y_pred)
print('R-square score before feature selection:', r2_score_before)

#R-square score before feature selection: 0.9027043809921144

ValueError: could not convert string to float: 'A-5602'

In [ ]:
#Now, I’ll use featurewiz to select the best features from the train set using a single line of code:

# use featurewiz to select the best features from the train set
selected_features, _ = fw.featurewiz(X_train.join(y_train),target=['Y1', 'Y2'], corr_limit=0.7, verbose=2)
print('Selected features:', selected_features)

#Selected features: ['X1', 'X7', 'X4', 'X3', 'X5']